In [18]:
import os, os.path
import sqlalchemy as sql
from whoosh import index, fields, analysis
from whoosh.filedb.filestore import FileStorage

from src import Config

In [5]:
config = Config()
index_dir = config['text_index_path']

In [6]:
storage = FileStorage(index_dir)
storage.clean()

In [7]:
schema = fields.Schema(
    # Identifier of this entry in the respective DB table
    id=fields.ID(stored=True),
    # Name (or names) of this journal/conference/publisher
    # NOT STORED, used only for scoring
    name=fields.NGRAMWORDS(),
    # Type of entry (journal/conference/publisher)
    # NOT STORED, used only for filtering
    type=fields.ID(),
    # Names of science domains for this entry
    # NOT STORED, used only for scoring
    domains=fields.KEYWORD(lowercase=True, commas=True, scorable=True),
)

In [8]:
os.makedirs(index_dir, exist_ok=True)
storage.create_index(schema)

FileIndex(FileStorage('./text_index'), 'MAIN')

In [25]:
engine = sql.create_engine(f"sqlite:///{config['db_file']}")
conn = engine.connect()

In [23]:
meta = sql.MetaData()
monographs = sql.Table('Monographs', meta, autoload_with=engine)


In [11]:
ix = index.open_dir(index_dir)
writer = ix.writer()

In [ ]:
# brok.
for r in conn.execute(sql.select(monographs)):
    print(r)